In [2]:
import sys
import os
import dspy 
from common.my_settings import MySettings  
from common.utils import md
from common.llm_client_factory import LlmClientFactory
from dspy_utils.dspy_helpers import md_dspy

settings = MySettings().get()

lm_gpt35 = dspy.LM('gpt-3.5-turbo', temperature=0.8, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
lm_gpt4omin = dspy.LM('gpt-4o-mini', temperature=0.9, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
dspy.configure(lm=lm_gpt4omin)

lm = lm_gpt4omin
lm('hello there')


Getting keys from environment variables


['Hello! How can I assist you today?']

In [3]:
lm_gpt4omin('Hows the weather')

["I don't have real-time weather data, but you can easily check the current weather by using a weather website, app, or asking a voice-activated assistant. If you tell me your location, I can help you find typical weather patterns for this time of year!"]

In [4]:
lm_gpt4omin.history

[{'prompt': 'hello there',
  'messages': None,
  'kwargs': {},
  'response': ModelResponse(id='chatcmpl-CCSH6mwmoWhJtsitOBewrrrqih6jG', created=1757084660, model='gpt-4o-mini-2024-07-18', object='chat.completion', system_fingerprint='fp_8bda4d3a2c', choices=[Choices(finish_reason='stop', index=0, message=Message(content='Hello! How can I assist you today?', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=9, prompt_tokens=9, total_tokens=18, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default', cache_hit=None),
  'outputs': ['Hello! How can I assist you today?'],
  'usage': {'completion_tokens': 9,
   

In [5]:
from datetime import datetime

print(datetime.now())

2025-09-05 17:04:23.525684


In [6]:
len(lm.history)
lm.history[-1].keys()

dict_keys(['prompt', 'messages', 'kwargs', 'response', 'outputs', 'usage', 'cost', 'timestamp', 'uuid', 'model', 'response_model', 'model_type'])

In [7]:
toxicity = dspy.Predict(
    dspy.Signature(
        "comment -> toxic: bool", 
        instructions="Mark as 'toxic' if the comment includes insults, harrasment or sarcastic comments"
    )
)

toxicity(comment="You are awesome").toxic

False

In [8]:
choose_meal = dspy.Predict(
    dspy.Signature(
    "meal_suggestion -> description",
    instructions="Given a meal suggestion come up with a meal and desribe it in 20 words, make it sound enticing without going overboard"
))

md(choose_meal(meal_suggestion="South African").description)


Savor a traditional South African bobotie, a spiced minced meat dish topped with creamy egg custard, served with fragrant yellow rice.

In [9]:
sentence = "Today is rainy and gloomy"

classify = dspy.Predict("sentence -> sentiment: str") # change type: int, float, str, bool
classify(sentence=sentence).sentiment

'Negative'

In [10]:
argh = "Let's go to a bar tonight and listen to a band"
pirate_speak = dspy.Predict("normal_speak -> pirate_speak")
pirate_speak(normal_speak=argh).pirate_speak

'Arrr, let’s set sail to a tavern tonight and hear the shanties of a band!'

In [11]:
question = "What colour are plant leaves?"
reason_question = dspy.Predict("question -> answer")
md(reason_question(question=question))

Prediction(  
    answer='Plant leaves are primarily green due to the presence of chlorophyll, which is essential for photosynthesis. However, leaves can also be found in a variety of other colors, including red, yellow, orange, and purple, depending on the plant species and the presence of other pigments like carotenoids and anthocyanins.'  
)

In [12]:
from typing import Literal

class Emotion(dspy.Signature):
    """Classify emotion"""
    sentence: str = dspy.InputField()
    sentiment: Literal["sadness", "joy", "love", "fear", "surprised"]  = dspy.OutputField()

classify = dspy.Predict(Emotion)
classify(sentence="rainy day")

Prediction(
    sentiment='sadness'
)

https://dspy.ai/learn/programming/signatures/

In [13]:
class CheckCitationOfFaithfulness(dspy.Signature):
    """Verify that the text is based on the provided context."""
    context: str = dspy.InputField(desc="Facts here are assumed to be true")
    text: str = dspy.InputField()
    faithfulness: float = dspy.OutputField()
    evidence: dict[str, list[str]] = dspy.OutputField(desc="Supporting evidence for claims")
    counter_argument: dict[str, dict[str, str]] = dspy.OutputField(desc="Give a counter argument to this")

context = """
The 21-year-old made seven appearances for the Hammers and netted his only 
goal for them in a Europa League qualification round match against Andorran side 
FC Lustrains last season. Lee had two loan spells in League One last term, 
with Blackpool and then Colchester United. He scored twice for the U's but 
was unable to save them from relegation. 
The length of Lee's contract with the promoted Tykes has not been revealed. 
Find all the latest football transfers on our dedicated page.
"""
text = "Lee scored 3 goals for the Colchester United"

faithfulness = dspy.ChainOfThought(CheckCitationOfFaithfulness)
faithfulness(context=context, text=text)


Prediction(
    reasoning='The text states that Lee scored 3 goals for Colchester United. However, the context clearly mentions that he scored only twice for Colchester. Therefore, the statement in the text is inaccurate and not faithful to the provided context.',
    faithfulness=0.0,
    evidence={'scored_for_Colchester_United': ["He scored twice for the U's but was unable to save them from relegation."]},
    counter_argument={'Lee_scored_goals': {'claim': 'Lee scored 3 goals for Colchester United', 'refutation': 'The context states that he scored only twice for Colchester United.'}}
)

# Custom Types

In [ ]:
class shop_to_obtain(dspy.Signature):
    """ Suggest a shop to obtain an item"""
    item: str = dspy.InputField(desc="An item to obtain")
    shop_name: str = dspy.OutputField(desc="A shop where the item can be obtained")
    address: str = dspy.OutputField(desc="The address of the shop")
    opening_hours: str = dspy.OutputField(desc="The opening hours of the shop")

class Ingredient(dspy.Signature):
    """ Describe a meal ingredient"""
    ingredient: str = dspy.InputField(desc="An ingredient")
    description: str = dspy.OutputField(desc="A description of the ingredient in 20 words")
    taste_profile: str = dspy.OutputField(desc="The taste profile of the ingredient in 10 words")
    shop: shop_to_obtain = dspy.OutputField(desc="A shop to obtain the ingredient")
    

class Meal(dspy.Signature):
    """Suggest a meal based on the cuisine"""
    meal_suggestion: str = dspy.InputField(desc="Suggest a meal")
    meal: str = dspy.OutputField(desc="A meal that fits the cuisine")
    description: str = dspy.OutputField(desc="A description of the meal in 20 words")
    ingredients: list[Ingredient] = dspy.OutputField(desc="List of ingredients for the meal")

meal = dspy.Predict(Meal)
md(meal(meal_suggestion="Italian"))


Prediction(  
    meal='Spaghetti Carbonara',  
    description='A classic Italian pasta dish featuring spaghetti coated in a creamy sauce made from eggs, cheese, pancetta, and pepper.',  
    ingredients=[Ingredient(ingredient='Spaghetti', description='Long, thin pasta made from durum wheat, ideal for various sauces and cooking methods.', taste_profile='Nutty, chewy, and absorbs flavors well.', shop=shop_to_obtain(item='Spaghetti', shop_name='Pasta Shop', address='123 Pasta Lane, Food City', opening_hours='9 AM - 8 PM')), Ingredient(ingredient='Pancetta', description='Italian cured meat made from pork belly, providing a rich and savory flavor to dishes.', taste_profile='Savory, salty, and slightly sweet.', shop=shop_to_obtain(item='Pancetta', shop_name='Meat Market', address='456 Meat Ave, Food City', opening_hours='8 AM - 6 PM')), Ingredient(ingredient='Eggs', description='Versatile protein-rich food often used in cooking, providing creaminess and richness to dishes.', taste_profile='Rich, creamy, and slightly savory.', shop=shop_to_obtain(item='Eggs', shop_name='Farm Fresh', address='789 Farm Rd, Food City', opening_hours='7 AM - 7 PM')), Ingredient(ingredient='Parmesan cheese', description='A hard, aged cheese with a strong flavor, commonly sprinkled over pasta dishes for added depth.', taste_profile='Nutty, salty, and sharp.', shop=shop_to_obtain(item='Parmesan cheese', shop_name='Cheese Delights', address='321 Cheese Blvd, Food City', opening_hours='10 AM - 5 PM')), Ingredient(ingredient='Black pepper', description='A common spice known for its pungent flavor and heat, enhancing the taste of many dishes.', taste_profile='Pungent, spicy, and slightly woody.', shop=shop_to_obtain(item='Black pepper', shop_name='Spice Corner', address='654 Spice St, Food City', opening_hours='9 AM - 6 PM'))]  
)

# Images

In [22]:
from IPython.display import Image, display

class PictureSignature(dspy.Signature):
    """Output the dog breed of the picture"""
    image_1: dspy.Image = dspy.InputField(desc="An image of a dog")
    answer: str = dspy.OutputField(desc="The dog breed of the dog in the picture")

picture = dspy.Predict(PictureSignature)
md(picture(image_1=dspy.Image.from_url("https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U")))
md(display(Image(url="https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U")))

Prediction(  
    answer='Labrador Retriever'  
)

None

In [24]:
class MyPicture(dspy.Signature):
    """Describe the image in 20 words"""
    image_1: dspy.Image = dspy.InputField(desc="An image")
    description: str = dspy.OutputField(desc="Describe the image in 20 words")
    country: str = dspy.OutputField(desc="What country does it look like it's in?")
    time_of_day: str = dspy.OutputField(desc="What time of day is it?")

images = [
    "https://thfvnext.bing.com/th/id/OIP.SgJROEuAItyr77TAKXSUKQHaF7?cb=thfvnext&rs=1&pid=ImgDetMain&o=7&rm=3",
    "https://th.bing.com/th/id/R.13eea3b5c84adcea91409e15c03fe8f4?rik=RkKRFRv4b5Votw&pid=ImgRaw&r=0",
    "https://lushpalm.com/wp-content/uploads/2018/05/bali-surf-camps-15.jpg",
    "https://thfvnext.bing.com/th/id/OIF.ruMvqnNb7DCPUSaEdR9iMQ?cb=thfvnext&rs=1&pid=ImgDetMain&o=7&rm=3",
    "https://thfvnext.bing.com/th/id/OIP.c6Tbz7IbCn9bVXzXQSOqhgHaFN?cb=thfvnext&rs=1&pid=ImgDetMain&o=7&rm=3"
]

describe = dspy.Predict(MyPicture)
for i in images:
    md(describe(image_1=dspy.Image.from_url(i)))    
    display(Image(url=i))



Prediction(  
    description='A statue of a woman sitting on a bench, accompanied by a dog, in a public square setting.',  
    country='Unknown',  
    time_of_day='Daytime'  
)

Prediction(  
    description='A surfer skillfully rides a large wave, surrounded by misty tropical foliage under a cloudy sky.',  
    country='It looks like it could be in a tropical country, possibly Hawaii or Indonesia.',  
    time_of_day='The time of day appears to be early morning or late afternoon.'  
)

Prediction(  
    description='Luxurious villa with a swimming pool, sun loungers, and lush greenery under clear blue skies, perfect for relaxation.',  
    country='Unknown',  
    time_of_day='Daytime'  
)

Prediction(  
    description='A scenic river view with a green park, bridge, and city skyline under a bright blue sky with fluffy clouds.',  
    country='United States',  
    time_of_day='Daytime'  
)

Prediction(  
    description='A colorful assortment of fruits includes bananas, apples, grapes, lemons, strawberries, and kiwi in a woven basket.',  
    country='Not specified',  
    time_of_day='Not specified'  
)